In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))
from pyomo.environ import *
from pyomo.gdp import *

     |████████████████████████████████| 8.9 MB 4.2 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

In [ ]:
M= [[[0, 1, 1, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 1],
  [1, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 0],
  [0,0,0,0,0,0]],
 [[0, 1, 1, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 0],
  [0, 0, 0, 0, 0, 0],
  [0,0,0,0,0,0]],
 [[0, 1, 1, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 1],
  [1, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 0],
  [0,0,0,0,0,0]]]
Dur = [[12.96, 12.96, 19.44, 12.96, 12.96, 6.04, 0],
       [12.96, 12.96, 12.96, 12.96, 5.04, 0, 0],
       [12.96, 12.96, 12.96, 12.96, 12.96, 5.04, 0]]
I=3
J=6
K=6

In [ ]:
Processes = [[('Cavity Insert, SG Block Up'),('Cavity Insert, CNC Turning Bottom'),('Cavity Insert, CNC Turning Top'),('Cavity Insert, CNC Milling'),('Cavity Insert, Insert Identification'),('Cavity Insert, Inspection 1')],
              [('Cavity Sub Insert, SG Block Up'),('Cavity Sub Insert, CNC Turning Bottom'),('Cavity Sub Insert, CNC Turning Top'),('Cavity Sub Insert, Insert Identification'),('Cavity Sub Insert, Inspection'),('-')],
             [('Cavity Insert, SG Block Up'),('Cavity Insert, CNC Turning Bottom'),('Cavity Insert, CNC Turning Top'),('Cavity Insert, CNC Milling Bottom'),('Cavity Insert, Insert Identification'),('Cavity Insert, Inspection')]]

In [ ]:
Machines = ['Unit II Conventional', 'Unit II CNC Turning - 1', 'Unit II CNC Turning - 2', 'Unit II CNC Milling - 1', 'Unit II CNC Milling - 2', '-']

In [ ]:

m = ConcreteModel()
m.Jobs= range(I)
m.Op = range(J)
m.Mach = range(K)
 

m.Bin = Var(m.Jobs, m.Op, m.Mach, domain=Boolean)

m.Bin2 = Var(m.Jobs, m.Op, m.Jobs, m.Op, m.Mach, domain=Boolean)

m.S = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeReals)

m.C = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeReals)
m.Ck = Var(m.Mach, domain=NonNegativeReals)
m.Ms = Var(domain=NonNegativeReals)

m.X = Var(m.Jobs, m.Op, m.Mach, domain=Boolean)


m.First = ConstraintList()#....................................(1)
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.First.add(M[i][j][k]+m.Bin[i,j,k]<=1)
for i in m.Jobs:
  for j in m.Op:
    m.First.add(sum(m.Bin[i,j,k] for k in m.Mach)==1)

m.Sec = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Sec.add(m.S[i,j,k]+m.C[i,j,k]<=m.Bin[i,j,k]*5000)
m.Third = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      if k==2:
        m.Third.add(m.C[i,j,k]>=m.S[i,j,k]+Dur[i][j]+8-(1-m.Bin[i,j,k])*5000)
      if  k==4:
        m.Third.add(m.C[i,j,k]>=m.S[i,j,k]+Dur[i][j]+8-(1-m.Bin[i,j,k])*5000)
      else:
        m.Third.add(m.C[i,j,k]>=m.S[i,j,k]+Dur[i][j]-(1-m.Bin[i,j,k])*5000)

m.Fourth=ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for i2 in m.Jobs:
      if i<i2:
        for j2 in m.Op:
          for k in range(0,6):
            m.Fourth.add(m.S[i,j,k]>=m.C[i2,j2,k]-(m.Bin2[i,j,i2,j2,k])*5000)
            m.Fourth.add(m.S[i2,j2,k]>=m.C[i,j,k]-(1-m.Bin2[i,j,i2,j2,k])*5000)

            

m.Sixth=ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    if j>0:
      m.Sixth.add(sum(m.S[i,j,k] for k in m.Mach)>=sum(m.C[i,j-1,k] for k in m.Mach))


m.Seventh = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Seventh.add(m.Ms>=m.Ck[k])

m.Eight = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Seventh.add(m.Ck[k]>=m.C[i,j,k])



m.d = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeIntegers)
m.h = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeReals)

m.Con = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      if k==2:
        m.Con.add(24*m.d[i,j,k]+m.h[i,j,k]==m.S[i,j,k])
        m.Con.add(m.h[i,j,k]>=0)
        m.Con.add(m.h[i,j,k]<=3.04)
      if k==4:
        m.Con.add(24*m.d[i,j,k]+m.h[i,j,k]==m.S[i,j,k])
        m.Con.add(m.h[i,j,k]>=0)
        m.Con.add(m.h[i,j,k]<=3.04)
#m.Ninth = ConstraintList()
#for i in m.Jobs:
  #for j in m.Op:
    #for k in m.Mach:
      #m.Ninth.add(m.S[i,j,k]>=m.Bin[i,j,k]*a2[k])





m.obj = Objective(expr = m.Ms, sense=minimize)

SolverFactory('cbc').solve(m).write()




# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 106.84
  Upper bound: 106.84
  Number of objectives: 1
  Number of constraints: 243
  Number of variables: 152
  Number of binary variables: 756
  Number of integer variables: 792
  Number of nonzeros: 1
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 2.66
  Wallclock time: 2.72
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and 

In [ ]:
def dat(k):
  a=k//1
  d=a//24
  t=a%24
  m=(k%1)*60
  return [d+1, t, m]
A=[]

In [ ]:

for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      if m.Bin[i,j,k]()==1:
        if Dur[i][j]>0:
          A.append([Processes[i][j], dat(m.S[i,j,k]()), dat(m.C[i,j,k]()), Machines[k]])
A


[['Cavity Insert, SG Block Up',
  [2.0, 1.0, 55.2000000000001],
  [2.0, 14.0, 52.80000000000015],
  'Unit II Conventional'],
 ['Cavity Insert, CNC Turning Bottom',
  [2.0, 14.0, 52.80000000000015],
  [3.0, 3.0, 50.400000000000205],
  'Unit II CNC Turning - 1'],
 ['Cavity Insert, CNC Turning Top',
  [3.0, 3.0, 50.400000000000205],
  [3.0, 23.0, 16.800000000000068],
  'Unit II CNC Turning - 1'],
 ['Cavity Insert, CNC Milling',
  [3.0, 23.0, 16.800000000000068],
  [4.0, 12.0, 14.399999999999693],
  'Unit II CNC Milling - 1'],
 ['Cavity Insert, Insert Identification',
  [4.0, 15.0, 50.400000000000205],
  [5.0, 4.0, 47.99999999999983],
  '-'],
 ['Cavity Insert, Inspection 1',
  [5.0, 4.0, 47.99999999999983],
  [5.0, 10.0, 50.400000000000205],
  '-'],
 ['Cavity Sub Insert, SG Block Up',
  [1.0, 12.0, 57.60000000000005],
  [2.0, 1.0, 55.2000000000001],
  'Unit II Conventional'],
 ['Cavity Sub Insert, CNC Turning Bottom',
  [2.0, 1.0, 55.2000000000001],
  [2.0, 22.0, 52.80000000000015],
  'Uni

In [ ]:
from tabulate import tabulate

In [ ]:
b = ['Process', 'Start (Day, Hour, Minute)', 'Stop(Day, Hour, Minute)', 'Machine']
print(tabulate(A, headers=b, tablefmt="grid"))

df = pd.DataFrame(A, columns = ['Process', 'Start', 'Stop', 'Machine'])
df.to_excel(r'new.xlsx', index = False)

+------------------------------------------+---------------------------------+---------------------------------+-------------------------+
| Process                                  | Start (Day, Hour, Minute)       | Stop(Day, Hour, Minute)         | Machine                 |
+==========================================+=================================+=================================+=========================+
| Cavity Insert, SG Block Up               | [2.0, 1.0, 55.2000000000001]    | [2.0, 14.0, 52.80000000000015]  | Unit II Conventional    |
+------------------------------------------+---------------------------------+---------------------------------+-------------------------+
| Cavity Insert, CNC Turning Bottom        | [2.0, 14.0, 52.80000000000015]  | [3.0, 3.0, 50.400000000000205]  | Unit II CNC Turning - 1 |
+------------------------------------------+---------------------------------+---------------------------------+-------------------------+
| Cavity Insert, CNC Turnin

# New Section

In [ ]:
Processes = [[('Core Insert, SG Block up'),('Core Insert, BHT CNC Turning Bottom Setting'),('Core Insert, BHT CNC Turning Top Setting'),('Core Insert, Conventional Turning'),('Core Insert, Insert Identification'),('Core Insert, Inspection'),('-')],
             [('Cooling  Insert, SG Block up'),('Cooling  Insert, CNC Turning Bottom Setting'),('Cooling  Insert, CNC Turning Top Setting'),('Cooling  Insert, CNC Milling Top Setting'),('Cooling  Insert, CNC Milling Bottom Setting'),('Cooling  Insert, Insert Identification'),('Cooling  Insert, Inspection'),],
             [('Core Sub Insert, SG Block up'),('Core Sub Insert, CNC Turning Bottom Setting'),('Core Sub Insert, CNC Turning Top Setting'),('Core Sub Insert, Conventional Milling'),('Core Sub Insert, Insert Identification'),('Core Sub Insert, Inspection'),('-')]]

In [ ]:
a2=[m.Ck[i]() for i in m.Mach]
a2

[38.88, 71.28, 68.96, 84.24, 0.0, 105.84]

In [ ]:

M=[[[0, 1, 1, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
   [1, 0, 0, 1, 1, 1],
   [1, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 0],
   [0,0,0,0,0,0]],
   [[0, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 1],
  [1, 1, 1, 0, 0, 1],
  [1, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 0]],
  [[0, 1, 1, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 1],
  [1, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 0],
   [0,0,0,0,0,0]]]



    


Dur=[[12.96, 25.92, 25.92, 12.96, 12.96, 5.04, 0],
     [12.96, 12.96, 12.96, 12.96, 19.44, 12.96, 5.04],
     [12.96, 12.96, 12.96, 12.96, 12.96, 5.04, 0]]
I=3
J=7
K=6


In [ ]:
m = ConcreteModel()
m.Jobs= range(I)
m.Op = range(J)
m.Mach = range(K)


m.Bin = Var(m.Jobs, m.Op, m.Mach, domain=Boolean)

m.Bin2 = Var(m.Jobs, m.Op, m.Jobs, m.Op, m.Mach, domain=Boolean)

m.S = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeReals)

m.C = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeReals)
m.Ck = Var(m.Mach, domain=NonNegativeReals)
m.Ms = Var(domain=NonNegativeReals)

m.X = Var(m.Jobs, m.Op, m.Mach, domain=Boolean)


m.First = ConstraintList()#....................................(1)
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.First.add(M[i][j][k]+m.Bin[i,j,k]<=1)
for i in m.Jobs:
  for j in m.Op:
    m.First.add(sum(m.Bin[i,j,k] for k in m.Mach)==1)

m.Sec = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Sec.add(m.S[i,j,k]+m.C[i,j,k]<=m.Bin[i,j,k]*5000)
m.Third = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      if k==2:
        m.Third.add(m.C[i,j,k]>=m.S[i,j,k]+Dur[i][j]+8-(1-m.Bin[i,j,k])*5000)
      if  k==4:
        m.Third.add(m.C[i,j,k]>=m.S[i,j,k]+Dur[i][j]+8-(1-m.Bin[i,j,k])*5000)
      else:
        m.Third.add(m.C[i,j,k]>=m.S[i,j,k]+Dur[i][j]-(1-m.Bin[i,j,k])*5000)

m.Fourth=ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for i2 in m.Jobs:
      if i<i2:
        for j2 in m.Op:
          for k in range(0,6):
            m.Fourth.add(m.S[i,j,k]>=m.C[i2,j2,k]-(m.Bin2[i,j,i2,j2,k])*5000)
            m.Fourth.add(m.S[i2,j2,k]>=m.C[i,j,k]-(1-m.Bin2[i,j,i2,j2,k])*5000)

            

m.Sixth=ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    if j>0:
      m.Sixth.add(sum(m.S[i,j,k] for k in m.Mach)>=sum(m.C[i,j-1,k] for k in m.Mach))


m.Seventh = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Seventh.add(m.Ms>=m.C[i,j,k])

m.Eight = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Seventh.add(m.Ck[k]>=m.C[i,j,k])



m.d = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeIntegers)
m.h = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeReals)

m.Con = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      if k==2:
        m.Con.add(24*m.d[i,j,k]+m.h[i,j,k]==m.S[i,j,k])
        m.Con.add(m.h[i,j,k]>=0)
        m.Con.add(m.h[i,j,k]<=3.04)
      if k==4:
        m.Con.add(24*m.d[i,j,k]+m.h[i,j,k]==m.S[i,j,k])
        m.Con.add(m.h[i,j,k]>=0)
        m.Con.add(m.h[i,j,k]<=3.04)



    


#m.Eigth = ConstraintList()
#m.Eigth.add(m.Ms>=sum(m.C[i,j,k] for i in m.Jobs for j in m.Op for k in m.Mach))

m.obj = Objective(expr = m.Ms, sense=minimize)

#SolverFactory('cbc').solve(m).write()




TypeError: ignored

In [ ]:
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      if m.Bin[i,j,k]()==1:
        if Dur[i][j]>0:
          print(i,j, m.S[i,j,k](),m.C[i,j,k](), k+1)
print(m.Ms())

0 0 144.0 192.0 2
0 1 192.0 246.0 1
0 2 246.0 282.0 3
0 3 282.0 318.0 6
0 4 318.0 332.0 5
0 5 332.0 368.0 5
1 0 200.0 240.0 2
1 1 240.0 276.0 2
1 2 318.0 354.0 6
1 3 368.0 382.0 5
2 0 144.0 180.0 1
2 1 246.0 282.0 1
2 2 282.0 318.0 3
2 3 318.0 354.0 3
2 4 354.0 390.0 6
2 5 390.0 404.0 5
1706.0


In [ ]:
a4=[m.Ck[i]() for i in m.Mach]
a4

[252.0, 216.0, 0.0, 288.0, 338.0, 324.0]

In [ ]:
m = ConcreteModel()
m.Jobs= range(I)
m.Op = range(J)
m.Mach = range(K)

m.Bin = Var(m.Jobs, m.Op, m.Mach, domain=Boolean)

m.Bin2 = Var(m.Jobs, m.Op, m.Jobs, m.Op, m.Mach, domain=Boolean)

m.S = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeIntegers)

m.C = Var(m.Jobs, m.Op, m.Mach, domain=NonNegativeIntegers)

m.Ms = Var(domain=Integers)

m.First = ConstraintList()#....................................(1)
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.First.add(M[i][j][k]+m.Bin[i,j,k]<=1)
for i in m.Jobs:
  for j in m.Op:
    m.First.add(sum(m.Bin[i,j,k] for k in m.Mach)==1)

m.Sec = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Sec.add(m.S[i,j,k]+m.C[i,j,k]<=m.Bin[i,j,k]*5000)
m.Third = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Sec.add(m.C[i,j,k]>=m.S[i,j,k]+Dur[i][j]-(1-m.Bin[i,j,k])*5000)

m.Fourth=ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for i2 in m.Jobs:
      if i<i2:
        for j2 in m.Op:
          for k in m.Mach:
            m.Fourth.add(m.S[i,j,k]>=m.C[i2,j2,k]-(m.Bin2[i,j,i2,j2,k])*5000)

m.Fifth=ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for i2 in m.Jobs:
      if i<i2:
        for j2 in m.Op:
          for k in m.Mach:
            m.Fifth.add(m.S[i2,j2,k]>=m.C[i,j,k]-(1-m.Bin2[i,j,i2,j2,k])*5000)

m.Sixth=ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    if j>0:
      m.Sixth.add(sum(m.S[i,j,k] for k in m.Mach)>=sum(m.C[i,j-1,k] for k in m.Mach))


m.Seventh = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Seventh.add(m.Ms>=m.C[i,j,k])

m.Ninth = ConstraintList()
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      m.Ninth.add(m.S[i,j,k]>=m.Bin[i,j,k]*a4[k])
    


#m.Eigth = ConstraintList()
#m.Eigth.add(m.Ms>=sum(m.C[i,j,k] for i in m.Jobs for j in m.Op for k in m.Mach))

m.obj = Objective(expr = m.Ms, sense=minimize)

SolverFactory('cbc').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 514.0
  Upper bound: 514.0
  Number of objectives: 1
  Number of constraints: 376
  Number of variables: 210
  Number of binary variables: 756
  Number of integer variables: 973
  Number of nonzeros: 1
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 43.8
  Wallclock time: 44.86
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and b

In [ ]:
for i in m.Jobs:
  for j in m.Op:
    for k in m.Mach:
      if m.Bin[i,j,k]()==1:
        if Dur[i][j]>0:
          print(i,j, m.S[i,j,k](),m.C[i,j,k](), k+1)

0 0 0.0 36.0 2
0 1 36.0 90.0 1
0 2 108.0 144.0 4
0 3 144.0 180.0 6
0 4 180.0 194.0 5
0 5 194.0 230.0 5
1 0 72.0 108.0 2
1 1 108.0 144.0 2
1 2 180.0 216.0 6
1 3 216.0 230.0 5
2 0 0.0 36.0 1
2 1 36.0 72.0 2
2 2 72.0 108.0 4
2 3 144.0 180.0 4
2 4 180.0 216.0 6
2 5 216.0 230.0 5
